# Imports

In [1]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 660 µs (started: 2023-01-03 14:24:29 +00:00)


In [2]:
import numpy as np
import math
import statistics
import random
import secrets
from numpy.random import default_rng

time: 25.3 ms (started: 2023-01-03 14:24:29 +00:00)


# Global Variables

In [3]:
POPULATION = []
NEW_POPULATION = []
length_population = 100
length_chromosome = 12
rate_crossover = 90
rate_mutation = 50
MinLT = 1
MaxLT = 3

time: 7.55 ms (started: 2023-01-03 14:24:29 +00:00)


# Class

In [4]:
class Chromosome:
  score = 0
  idade = 0
  tempo_vida = 0
  
  def __init__(self,schema):
    self.schema = schema
  
  def __str__(self):
    toString = ''
    for ind in self.schema:
      toString += ind  
    return toString

time: 4.03 ms (started: 2023-01-03 14:24:29 +00:00)


# Selection

In [5]:
def avaregePopulation(population):
  sum = 0
  for i in population:
    sum += i.score
  
  if sum == 0:
    return 0
  else:
    return sum/ len(population)

time: 13.2 ms (started: 2023-01-03 14:24:30 +00:00)


In [6]:
def tempoVidaMedio(population):
  sum = 0
  for i in population:
    sum += i.tempo_vida
  
  return sum/ len(population)

time: 7.68 ms (started: 2023-01-03 14:24:30 +00:00)


In [7]:
def minFuc(n , score, mediaPopulation):
  p = MinLT + round(n * (score/mediaPopulation))
  return p

time: 1.14 ms (started: 2023-01-03 14:24:30 +00:00)


In [8]:
def vida(popu):
  n = tempoVidaMedio(popu)
  mediaPopulation = avaregePopulation(popu)

  for cromo in popu:
    res = minFuc(n,cromo.score, mediaPopulation)
    cromo.tempo_vida = min(res, MaxLT)


time: 1.43 ms (started: 2023-01-03 14:24:30 +00:00)


In [9]:
def selection(population, new_population):

  result = []
  merged_list = []
  merged_list.extend(population)
  merged_list.extend(new_population)

  for ind in merged_list:
    if ind.idade < ind.tempo_vida:
      result.append(ind)

  result.sort(key=lambda schema: schema.score, reverse = True)

  return result

time: 4.29 ms (started: 2023-01-03 14:24:30 +00:00)


# Mutation

In [10]:
def mutation(population):
  array = []

  for ind in population:
    array_2 = []
    for i,ind_schema in enumerate(ind.schema):
      yes = np.random.randint(0,100)

      if yes <= rate_mutation:
        if ind_schema == '0':
          array_2.append('1')
        elif ind_schema == '1':
          array_2.append('0')
      else:
        array_2.append(ind_schema)
  
    array.append(Chromosome(array_2))

  return array

time: 2.23 ms (started: 2023-01-03 14:24:30 +00:00)


# CrossOver

In [11]:
def crossOver(population):
  roulette = []
  sum_total_population = sum(ind.score for ind in population)
  
  for ind in population:
    length_individual = round((ind.score/sum_total_population)*100)
    
    for i in range(length_individual):
      roulette.append(ind.schema)
  
  max = len(roulette) -1
  
  while len(NEW_POPULATION) < length_population:
    father = roulette[np.random.randint(0,max)]
    mother = roulette[np.random.randint(0,max)]   
    probability_crossover = np.random.randint(0,100)

    if probability_crossover <= rate_crossover:
      if father != mother:
        child = []
        cut = np.random.randint(1,length_chromosome)
        child.append(father[:cut] + mother[cut:])
        child.append(mother[:cut] + father[cut:])
        
        for downward in child: 
          NEW_POPULATION.append(Chromosome(downward))

time: 4.2 ms (started: 2023-01-03 14:24:30 +00:00)


# Score

In [12]:
def score(population_test):
  for ind in population_test:
    ind.score= ind.schema.count('1')

time: 955 µs (started: 2023-01-03 14:24:30 +00:00)


# Init Population

In [13]:
def random(): 
  array2 = []

  rng = default_rng()
  numbers = rng.choice(range(0, 2), size= 2, replace=False)
  
  array2.append(str(numbers[0]))
  array2.append(str(numbers[1]))
  
  return array2

time: 3.78 ms (started: 2023-01-03 14:24:30 +00:00)


In [14]:
def init_population():
  for _ in range(length_population):
    array = []
    for _ in range(int(length_chromosome/2)):
      array.extend(random())
    POPULATION.append(Chromosome(array))


time: 1.41 ms (started: 2023-01-03 14:24:30 +00:00)


# Population Diversity

In [15]:
def population_diversity(population):
  array_2 = []
  for i in population:
    array_2.append(str(i))
  
  return (len(set(array_2))/len(population))*100

time: 13.9 ms (started: 2023-01-03 14:24:30 +00:00)


# Main

In [16]:
count = 0
array_geracao = []
list_more_adaptado = []
array_diversidade = []

while count < 1:
  flag = False
  generation = 0
  POPULATION.clear()
  NEW_POPULATION.clear()
  init_population()
  score(POPULATION)
  vida(POPULATION)

  while True:
    score(POPULATION)
    array_diversidade.append(population_diversity(POPULATION))
    crossOver(POPULATION)
    score(NEW_POPULATION)
    NEW_POPULATION = mutation(NEW_POPULATION)
    score(NEW_POPULATION)
    vida(NEW_POPULATION)
    POPULATION = selection(POPULATION,NEW_POPULATION)
    NEW_POPULATION.clear()
    
    if POPULATION[0].score == 12:
      flag = True
    
    for ind in POPULATION:
      ind.idade += 1

    # list_more_adaptado.append(POPULATION[0].score)

    if flag:
      print("===================================================================")
      print(f'Individuo: {POPULATION[0].schema} e o score dele {POPULATION[0].score} geracao {generation}')
      print("===================================================================")
      array_geracao.append(generation)
      break
      
    generation += 1
  count += 1
  
# print(f'media das 100 geracao {statistics.mean(array_geracao)}')

Individuo: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'] e o score dele 12 geracao 87
time: 1.09 s (started: 2023-01-03 14:24:30 +00:00)


In [17]:
print(array_geracao)

[87]
time: 748 µs (started: 2023-01-03 14:24:31 +00:00)


In [18]:
# print(list_more_adaptado)

time: 512 µs (started: 2023-01-03 14:24:31 +00:00)


In [19]:
print(array_diversidade)

[48.0, 72.5, 100.0, 98.0, 99.0, 99.0, 99.0, 100.0, 99.0, 99.0, 98.0, 99.0, 99.0, 99.0, 100.0, 98.0, 98.0, 98.0, 100.0, 100.0, 100.0, 100.0, 98.0, 100.0, 100.0, 99.0, 100.0, 99.0, 98.0, 100.0, 99.0, 100.0, 100.0, 99.0, 99.0, 100.0, 100.0, 100.0, 98.0, 100.0, 100.0, 98.0, 99.0, 100.0, 98.0, 100.0, 97.0, 99.0, 99.0, 99.0, 99.0, 98.0, 99.0, 99.0, 100.0, 99.0, 100.0, 95.0, 100.0, 98.0, 99.0, 100.0, 99.0, 98.0, 100.0, 100.0, 99.0, 100.0, 99.0, 99.0, 99.0, 98.0, 99.0, 100.0, 100.0, 99.0, 97.0, 100.0, 98.0, 97.0, 99.0, 100.0, 99.0, 98.0, 99.0, 98.0, 99.0, 97.0]
time: 875 µs (started: 2023-01-03 14:24:31 +00:00)
